<a href="https://colab.research.google.com/github/41371130H/PL-Repo/blob/main/HW1_%E6%97%A5%E5%B8%B8%E6%94%AF%E5%87%BA%E9%80%9F%E7%AE%97%E8%88%87%E5%88%86%E6%94%A4_Gradio.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **操作說明**
  有三種動作可供選擇



---



##   **新增資料**

  ***日期時間自動輸入***
  1.  輸入項目（1: 早餐, 2: 午餐, 3: 晚餐, 4: 飲料, 5: 交通, 6: 就學用品, 7: 日常用品）
  2.  輸入品項
  3.  輸入金額
  4.  是否要AA分攤（yes or no）

          yes：自動除以輸入之人數並顯示實付金額在輸出區與試算表


---


##   **刪除資料**

依照提供之表格輸入要刪除的資料列號


---


##   **統整資料**

顯示所有金額總和、各項目金額排行



---



**Google Sheet**：https://docs.google.com/spreadsheets/d/1KhaWfYSh2fGRmwC8SkZG8GdbTshYql4QX-hknk0ON84/edit?usp=sharing







In [13]:
from google.colab import auth
auth.authenticate_user()
import gspread
from google.auth import default
creds, _ = default()
gc = gspread.authorize(creds)

import pandas as pd
import datetime
import pytz
import gradio as gr
import ipywidgets as widgets
from IPython.display import display, clear_output

# 連線到 Google Sheet
gsheets = gc.open_by_url('https://docs.google.com/spreadsheets/d/1KhaWfYSh2fGRmwC8SkZG8GdbTshYql4QX-hknk0ON84/edit?usp=sharing')
worksheet = gsheets.worksheet('工作表1')


# ========== 功能 1：新增資料 ==========
def add_record(category, item, amount, aa, people):
    import datetime, pytz

    tz = pytz.timezone("Asia/Taipei")
    now = datetime.datetime.now(tz)
    date_str = now.strftime("%Y-%m-%d")
    time_str = now.strftime("%H:%M")

    # 是否AA分攤
    if aa == "yes" and people > 0:
        amount = amount // people

    new_data = pd.DataFrame([{
        "日期": date_str,
        "時間": time_str,
        "項目": category,  # 直接用選項文字
        "品項": item,
        "金額": amount
    }])

    data_to_write = new_data.values.tolist()
    worksheet.append_rows(values=data_to_write, value_input_option="USER_ENTERED")

    # # 讀取最新 5 筆
    # all_data = worksheet.get_all_values()
    # df = pd.DataFrame(all_data[1:], columns=all_data[0])
    # df["金額"] = pd.to_numeric(df["金額"], errors="coerce")
    # last5 = df.tail(5).reset_index(drop=True)

    return f"✅ 已新增資料：{category} - {item} - {amount} 元", show_records()

# ========== 功能 2：刪除資料 ==========
def show_records():
    all_data = worksheet.get_all_values()
    df = pd.DataFrame(all_data[1:], columns=all_data[0])
    df["金額"] = pd.to_numeric(df["金額"], errors="coerce")
    # 加入列號欄位
    return df.reset_index(drop=True).reset_index().rename(columns={"index": "列號"})

def delete_record(row_number):
    # row_number 是從 0 開始的 index，所以要 +2 才是 Google Sheet 的實際 row
    worksheet.delete_rows(row_number + 2)
    return f"🗑️ 已刪除第 {row_number} 列資料", show_records()


# ========== 功能 3：統整資料 ==========
def summarize_records():
    all_data = worksheet.get_all_values()
    df = pd.DataFrame(all_data[1:], columns=all_data[0])
    df["金額"] = pd.to_numeric(df["金額"], errors="coerce")

    total_amount = df["金額"].sum()
    category_summary = (
        df.groupby("項目")["金額"]
        .sum()
        .sort_values(ascending=False)
        .reset_index()
    )

    return total_amount, category_summary


# ========== Gradio 介面 ==========
with gr.Blocks() as demo:
    gr.Markdown("# 💵 Google Sheet 記帳系統")

    # 🔹 先把刪除資料的表格放在外層（變成全域元件）
    records_table = gr.Dataframe(
        value=show_records(),
        label="📋 目前紀錄",
        interactive=False
    )


    with gr.Tab("新增資料"):
        category = gr.Dropdown(
            ["早餐", "午餐", "晚餐", "飲料", "交通", "就學用品", "日常用品", "其他"],
            label="項目"
        )
        item = gr.Textbox(label="品項")
        amount = gr.Number(label="金額", precision=0)
        aa = gr.Radio(["yes", "no"], label="是否AA分攤", value="no")
        people = gr.Number(label="分攤人數", value=1, precision=0, visible=False)  # 🔹 一開始隱藏
        add_btn = gr.Button("新增資料")
        add_output = gr.Textbox(label="結果")

        # 🔹 定義一個函式來控制分攤人數顯示/隱藏
        def toggle_people(aa_choice):
          if aa_choice == "yes":
            return gr.update(visible=True, value=1)
          else:
            return gr.update(visible=False, value=1)

        # 當選擇是否AA分攤時，更新「分攤人數」欄位
        aa.change(fn=toggle_people, inputs=aa, outputs=people)

        add_btn.click(
            fn=add_record,
            inputs=[category, item, amount, aa, people],
            outputs=[add_output, records_table]
        )

    with gr.Tab("刪除資料"):
      row_number = gr.Number(label="要刪除的列號 (從 0 開始)", precision=0)
      delete_btn = gr.Button("刪除資料")
      delete_output = gr.Textbox(label="結果")

      # 點「刪除資料」會刪掉指定列，並回傳更新後的表格
      delete_btn.click(fn=delete_record, inputs=row_number, outputs=[delete_output, records_table])



    with gr.Tab("統整資料"):
        summary_btn = gr.Button("統整資料")
        total_output = gr.Textbox(label="💰 總金額")
        summary_output = gr.Dataframe(headers=["項目", "金額"], label="📊 各項目排行")
        summary_btn.click(fn=summarize_records, outputs=[total_output, summary_output])


demo.launch()



It looks like you are running Gradio on a hosted Jupyter notebook, which requires `share=True`. Automatically setting `share=True` (you can turn this off by setting `share=False` in `launch()` explicitly).

Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
* Running on public URL: https://246daead5edf356242.gradio.live

This share link expires in 1 week. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)
